In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 281 ms (started: 2021-09-05 13:24:03 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #3 Sept 05, 2021'

#git push origin master
git push

脚步

导入所有必要的库以加载我们的数据
定义并初始化神经网络
指定数据将如何通过您的模型
[可选] 通过模型传递数据进行测试深度学习使用人工神经网络（模型），这是由许多层互连单元组成的计算系统。 通过这些相互连接的单元传递数据，神经网络能够学习如何近似将输入转换为输出所需的计算。 在 PyTorch 中，可以使用 torch.nn 包构建神经网络。

# 介绍

PyTorch 提供了设计精美的模块和类，包括 torch.nn，以帮助您创建和训练神经网络。 nn.Module 包含层和返回输出的方法 forward(input)。

在这个秘籍中，我们将使用 torch.nn 定义一个用于 MNIST 数据集的神经网络。

# 步骤
* 导入所有必要的库以加载我们的数据
* 定义并初始化神经网络
* 指定数据将如何通过您的模型
* [可选] 通过模型传递数据进行测试

1. 导入必要的库来加载我们的数据

对于这个秘籍，我们将使用 torch 及其子公司 torch.nn 和 torch.nn.functional。

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

time: 694 ms (started: 2021-09-05 13:26:17 +08:00)


2. 定义并初始化神经网络

我们的网络将识别图像。 我们将使用 PyTorch 中内置的称为卷积的过程。 卷积将图像的每个元素添加到其局部邻居，由内核或小矩阵加权，这有助于我们从输入图像中提取某些特征（如边缘检测、锐度、模糊度等）。

定义模型的 Net 类有两个要求。 第一个是编写一个引用 nn.Module 的 `__init__` 函数。 这个函数是你在你的神经网络中定义全连接层的地方。

使用卷积，我们将定义我们的模型以获取 1 个输入图像通道，输出匹配我们代表数字 0 到 9 的 10 个标签的目标。该算法由您创建，我们将遵循标准的 MNIST 算法。

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # 第一个 2D 卷积层，接收 1 个输入通道（图像），
        # 输出32个卷积特征，平方核大小为3
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # 第二个 2D 卷积层，包含 32 个输入层，
        # 输出64个卷积特征，平方核大小为3
        self.conv2 = nn.Conv2d(32, 64, 3, 1)

        # 旨在确保相邻像素全为 0 或全为活动像素
        # 输入概率
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        # 第一个全连接层
        self.fc1 = nn.Linear(9216, 128)
        # 第二个全连接层，输出我们的 10 个标签
        self.fc2 = nn.Linear(128, 10)

my_nn = Net()
print(my_nn)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
time: 38.6 ms (started: 2021-09-05 13:29:10 +08:00)


我们已经完成了我们的神经网络的定义，现在我们必须定义我们的数据将如何通过它。

3. 指定数据将如何通过您的模型

当您使用 PyTorch 构建模型时，您只需定义前向函数，它将数据传递到计算图（即我们的神经网络）中。 这将代表我们的前馈算法。

您可以在前向函数中使用任何张量操作。

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    # x 代表我们的数据
    def forward(self, x):
        # 通过conv1传递数据
        x = self.conv1(x)
        # 在 x 上使用修正线性激活函数
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)

        # 在 x 上运行最大池化
        x = F.max_pool2d(x, 2)
        # 通过 dropout1 传递数据
        x = self.dropout1(x)
        # 用 start_dim=1 压平 x
        x = torch.flatten(x, 1)
        # 通过 fc1 传递数据
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        # 将 softmax 应用到 x
        output = F.log_softmax(x, dim=1)
        return output

time: 911 µs (started: 2021-09-05 13:33:10 +08:00)


4. [可选] 通过模型传递数据进行测试

为了确保我们收到我们想要的输出，让我们通过传递一些随机数据来测试我们的模型。

In [6]:
# 相当于一张随机的 28x28 图像
random_data = torch.rand((1, 1, 28, 28))

my_nn = Net()
result = my_nn(random_data)
print (result)

tensor([[-2.3510, -2.3655, -2.2650, -2.3701, -2.2393, -2.2745, -2.3689, -2.3049,
         -2.2212, -2.2797]], grad_fn=<LogSoftmaxBackward>)
time: 18.1 ms (started: 2021-09-05 13:34:02 +08:00)


这个结果张量中的每个数字都等于对随机张量关联的标签的预测。

恭喜！ 您已经在 PyTorch 中成功定义了一个神经网络。